In [ ]:
import torch
from torch.utils.data.dataset import Dataset
import math
import swanlab # 如果wandb对你来说获取困难
swanlab.login(api_key='ASzRe235nSSvVxYlaIBbq') # 替换为你的swanlab API key

In [ ]:
class SinModel(torch.nn.Module):
    def __init__(self,order=7):
        super(SinModel, self).__init__()
        self.w = torch.nn.Parameter(torch.randn(order+1, 1))
    def forward(self, x):
        x = torch.cat([x**i for i in range(self.w.shape[0])], dim=1)
        return x @ self.w

In [ ]:
class Sindataset(Dataset):
    def __init__(self, start=-1,end=1,len=1000):
        self.len =1000
        self.start = start
        self.end = end

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        x = torch.rand(1) * (self.end - self.start) + self.start
        y = torch.sin(x)
        return x, y
def get_ground_truth(order=7):
    w = []
    for i in range(order):
        # Taylor series coefficients for sin(x)
        w.append([0, 1][i % 2] * (1 / math.factorial(i)))

In [ ]:
batch_size = 1# 尝试小的batch size，之后改成大的
lr = 0.01
order = 7 # 多项式的阶数
model = SinModel(order=order)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.MSELoss()
train_dataset = Sindataset()
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
swanlab.init(project='sin', name='sin_model', config={
    'order': 7,
    'learning_rate': 0.01,
    'batch_size': 16,
    'epochs': 100
})

In [ ]:
for epoch in range(100):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        optimizer.zero_grad()
        outputs = SinModel()(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        swanlab.log({"step_loss": loss.item()}, step=epoch * len(trainloader) + i)
        print('Step loss:', loss.item())
    
    print(f'Epoch [{epoch+1}/100], Loss: {running_loss / len(trainloader):.4f}')
    swanlab.log({"epoch": epoch+1, "loss": running_loss / len(trainloader)})
print("model learned:",model.w)
print("Ground truth:",get_ground_truth(order=order))
